# Import Dependencies

In [1]:
import re
import pickle
import string
import numpy as np
import pandas as pd

# import spacy
# from spacy.lang.en.stop_words import STOP_WORDS
# from spacy.attrs import IS_ALPHA
# from spacy.lang.en import English
# from spacy import displacy, lemmatizer

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

import matplotlib.pyplot as plt

# from wordcloud import WordCloud
# from textwrap import wrap

import pyLDAvis
import pyLDAvis.sklearn

# Custom functions from .py files
from web_scrape import get_transcript
from preprocess import clean, preprocess

# Import/Preprocess Data

In [2]:
# Web scrape most recent speeches using our imported 'get_transcript'
goodyear_az = get_transcript('https://www.rev.com/blog/transcripts/donald-trump-rally-speech-transcript-goodyear-az-october-28')
bullhead_az = get_transcript('https://www.rev.com/blog/transcripts/donald-trump-rally-speech-transcript-bullhead-city-az-october-28')
omaha_ne = get_transcript('https://www.rev.com/blog/transcripts/donald-trump-rally-speech-transcript-omaha-ne-october-27')
wsalem_wi = get_transcript('https://www.rev.com/blog/transcripts/donald-trump-rally-speech-transcript-west-salem-wisconsin-october-27')
lansing_mi = get_transcript('https://www.rev.com/blog/transcripts/donald-trump-rally-speech-transcript-lansing-michigan-october-27')
martinsburg_pa = get_transcript('https://www.rev.com/blog/transcripts/donald-trump-rally-speech-transcript-martinsburg-pa-october-26')
lititz_pa = get_transcript('https://www.rev.com/blog/transcripts/donald-trump-rally-speech-transcript-lititz-pa-october-26')
allentown_pa = get_transcript('https://www.rev.com/blog/transcripts/donald-trump-rally-speech-transcript-allentown-pa-october-26')

In [3]:
# Create corpus for each state
az = [goodyear_az, bullhead_az]
ne = [omaha_ne]
wi = [wsalem_wi]
mi = [lansing_mi]
pa = [martinsburg_pa, lititz_pa, allentown_pa]

In [4]:
# Clean state corpuses
clean_az = clean(az)
clean_ne = clean(ne)
clean_wi = clean(wi)
clean_mi = clean(mi)
clean_pa = clean(pa)

In [5]:
# Preprocess our state corpuses
processed_az = preprocess(clean_az)
processed_ne = preprocess(clean_ne)
processed_wi = preprocess(clean_wi)
processed_mi = preprocess(clean_mi)
processed_pa = preprocess(clean_pa)

In [6]:
# Create corpus of all speeches
corpus = [goodyear_az, bullhead_az, omaha_ne, wsalem_wi, lansing_mi, martinsburg_pa, lititz_pa, allentown_pa]

In [7]:
# Clean corpus using our imported 'clean' function
clean_corpus = clean(corpus)

In [8]:
# Preprocess the corpus using our imported 'preprocess' function
processed_corpus = preprocess(clean_corpus)

# Doc-Term Matrix

In [27]:
# Countvectorizer, make dataframes

cv = CountVectorizer(stop_words='english')
cv_az = CountVectorizer(stop_words='english')
cv_ne = CountVectorizer(stop_words='english')
cv_wi = CountVectorizer(stop_words='english')
cv_mi = CountVectorizer(stop_words='english')
cv_pa = CountVectorizer(stop_words='english')

X = cv.fit_transform(processed_corpus)

X_az = cv_az.fit_transform(processed_az)
df_az = pd.DataFrame(X_az.toarray(), columns=cv_az.get_feature_names())

X_ne = cv_ne.fit_transform(processed_ne)
df_ne = pd.DataFrame(X_ne.toarray(), columns=cv_ne.get_feature_names())

X_wi = cv_wi.fit_transform(processed_wi)
df_wi = pd.DataFrame(X_wi.toarray(), columns=cv_wi.get_feature_names())

X_mi = cv_mi.fit_transform(processed_mi)
df_mi = pd.DataFrame(X_mi.toarray(), columns=cv_mi.get_feature_names())

X_pa = cv_pa.fit_transform(processed_pa)
df_pa = pd.DataFrame(X_pa.toarray(), columns=cv_pa.get_feature_names())

In [28]:
# Add 'State' column to each dataframe
df_az['State'] = [0] * len(df_az) # AZ
df_ne['State'] = [1] * len(df_ne) # NE
df_wi['State'] = [2] * len(df_wi) # WI
df_mi['State'] = [3] * len(df_mi) # MI
df_pa['State'] = [4] * len(df_pa) # PA

In [29]:
# Concatenate the state dataframes into one and fillnas with 0
df = pd.concat([df_az, df_ne, df_wi, df_mi, df_pa])
df.fillna(0, inplace=True)

# LDA

In [34]:
# for TF DTM
lda = LatentDirichletAllocation(n_components=5, random_state=0)
lda.fit(X)

LatentDirichletAllocation(n_components=5, random_state=0)

In [35]:
# LDA Visualization for CountVectorizer
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda, X, cv)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.086292  0.101557       1        1  21.976809
4     -0.103340  0.205087       2        1  21.850165
3     -0.073705 -0.207174       3        1  21.507271
1     -0.040881 -0.130247       4        1  18.231010
0      0.304217  0.030777       5        1  16.434745, topic_info=         Term        Freq       Total Category  logprob  loglift
1298     know  442.000000  442.000000  Default  30.0000  30.0000
2030    right  397.000000  397.000000  Default  29.0000  29.0000
1715   people  337.000000  337.000000  Default  28.0000  28.0000
2693     year  281.000000  281.000000  Default  27.0000  27.0000
2417    think  273.000000  273.000000  Default  26.0000  26.0000
...       ...         ...         ...      ...      ...      ...
2166     shut   21.140897   25.229944   Topic5  -5.4408   1.6290
1576   nevada   23.266759   34.895221   Topic5  -5.3450   1.4004
501   country   36.140587  223.537674   Topic5  -4.9046  -0.0164
986       god   23.760979   40.280650   Topic5  -5.3240   1.2779
1554   nation   22.474661   66.345499   Topic5  -5.3796   0.7233

[276 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
36        4  0.908525  advantage
50        1  0.986138        ago
51        5  0.970002      agree
73        1  0.089575    amazing
73        5  0.895754    amazing
...     ...       ...        ...
2693      3  0.003553       year
2697      1  0.031165  yesterday
2697      4  0.966122  yesterday
2699      3  0.842385      young
2699      5  0.150426      young

[388 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 5, 4, 2, 1])